<a href="https://colab.research.google.com/github/Xvobs/UNIFESP-X-ray-Body-Part-Classifier-Project/blob/main/Projekt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pydicom as dicom
import matplotlib.pyplot as plt
import numpy as np
import csv

def draw_img_dcm(image_path):
    ds = dicom.dcmread(image_path)

    pixel_array_numpy = ds.pixel_array

    plt.imshow(pixel_array_numpy)

input_shape = [128, 128, 3]
from skimage.transform import resize

def dcm_to_pixel_array(image_path):
    ds = dicom.dcmread(image_path)
    data = ds.pixel_array
    # resize to new size
    resized_img = resize(data, (input_shape[0], input_shape[0]), anti_aliasing=True)
    return resized_img

# specify your image path
#image_path = '../input/unifesp-xray-bodypart-classification/train/train/train/1002/1.2.826.0.1.3680043.8.498.66151583660414380258303341865105351564/1.2.826.0.1.3680043.8.498.40285474650218685810081647255379256989/1.2.826.0.1.3680043.8.498.37634180084111795685928688942145555908-c.dcm'
#draw_img_dcm(image_path)
#image_path = '../input/unifesp-xray-bodypart-classification/train/train/train/1002/1.2.826.0.1.3680043.8.498.66151583660414380258303341865105351564/1.2.826.0.1.3680043.8.498.88520490971705323792465522631523753284/1.2.826.0.1.3680043.8.498.12781870272366327239431992873991536152-c.dcm'
#draw_img_dcm(image_path)

#Match labels in train.csv - train data
#Match labels in sample_submission.csv - test data
#SOPInstanceUID - Each SOPInstanceUID corresponds to a unique image.
#Target - In the training data, this represents the labels assigned to each sample.

##THERE CAN BE MULTIPLE CLASSES IN ONE PICTURE

#Lables:
#    Abdomen = 0
#    Ankle = 1
#    Cervical Spine = 2
#    Chest = 3
#    Clavicles = 4
#    Elbow = 5
#    Feet = 6
#    Finger = 7
#    Forearm = 8
#    Hand = 9
#    Hip = 10
#    Knee = 11
#    Lower Leg = 12
#    Lumbar Spine = 13
#    Others = 14
#    Pelvis = 15
#    Shoulder = 16
#    Sinus = 17
#    Skull = 18
#    Thigh = 19
#    Thoracic Spine = 20
#    Wrist = 21

def extract_name_from_path(path):
    path = path.replace('../input/unifesp-xray-bodypart-classification/train/train/train/*/*/*/',' ')
    name = path.split("-")[-2].split("/")[-1]
    return name

#load data
import glob

temp_data_paths = glob.glob('../input/unifesp-xray-bodypart-classification/train/train/train/*/*/*/*.dcm')

#3 columns to later assign the label
train_data = [[0 for col in range(3)] for row in range(len(temp_data_paths))]

#convert dcms to pixel arrays and save the file name in 2d array
i = 0
for path in temp_data_paths:
    train_data[i][0] = dcm_to_pixel_array(path)
    train_data[i][1] = extract_name_from_path(path)
    i += 1
    
#print(train_data[0][1])
#plt.imshow(train_data[0][0])

#sort train_data desc filename to easily match labels
train_data.sort(key=lambda x:x[1])

#get columns more easy
train_data = np.array(train_data)
#print(train_data[:,0])

label_dict = ['Abdomen', 'Ankle', 'Cervical_Spine', 'Chest', 'Clavicles', 'Elbow', 'Feet', 'Finger', 'Forearm', 'Hand', 'Hip', 'Knee', 'Lower_Leg','Lumbar_Spine','Others', 'Pelvis', 'Shoulder','Sinus', 'Skull','Thigh','Thoracic_Spine','Wrist']

import pandas
image_label_mapping = pandas.read_csv('../input/unifesp-xray-bodypart-classification/train.csv')

#sort image_label_mapping after file name (asc)
image_label_mapping.sort_values(by=['SOPInstanceUID'])

image_name=image_label_mapping['SOPInstanceUID']
temp_labels=np.array(image_label_mapping['Target'])

#replace labels with tags
labels = []
for s in temp_labels:
    split = s.split(' ')
    for i in range(0,len(split)-1):
        if(i < 1):
            labels.append(split[i].replace(split[i], label_dict[int(split[i])]))
        else:
            labels[(i-1)-i] = labels[i] + ' , ' + label_dict[int(split[i])]
#print(labels)

#Tokenize
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters=',')

tokenizer.fit_on_texts(labels)

label_seq = tokenizer.texts_to_sequences(labels)

label_length=len(tokenizer.word_index)+1
print(tokenizer.word_index)


#split into train and test data while combining labels and data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data[:,0], labels, test_size=0.2, random_state=1)

#print(x_train[0])
plt.imshow(x_train[0])
print(y_train[0])
#print(y_train[0])
#print(x_train)
#print(y_train)

#catergorize
from tensorflow.keras.utils import to_categorical

labels=[to_categorical(label,num_classes=label_length,dtype='float32').sum(axis=0)[1:] for label in label_seq]

from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG19

model=models.Sequential()
conv_base = VGG19(weights='imagenet',
include_top=False,
input_shape=input_shape)
conv_base.trainable=False
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Activation('relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(22, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
  loss='binary_crossentropy', metrics=['accuracy'])

def get_dataset(x,y,batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    return dataset

steps_per_epoch=int(len(x_train)/32)
validation_step=int(len(x_test)/32)

train_ds=get_dataset(x_train,np.float32(y_train))
test_ds=get_dataset(x_test,np.float32(y_test))

history = model.fit(train_ds,
                    epochs=50,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_step,   
                    validation_data=test_ds)

#to decode predictions
def tags_mapping(one_hot_encoding):
    values = one_hot_encoding.round()
    tags = [tokenizer.index_word[i+1] for i in range(len(values)) if values[i] == 1.0]
    return tags

img_id=range(len(x_train)-1)
img,label=x_train[img_id],y_train[img_id]

prediction=model.predict(img,steps=1)[0]

prediction_tags=tags_mapping(prediction)

original_tags=tags_mapping(label)
image = x_train[img_id]
plt.imshow(image)